# utils

> Tools to help understand what code is doing. 

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Literal
import psutil
import rich
import time
import multiprocessing
import tracemalloc
import pandas as pd
import numpy as np
import datetime

In [ ]:
#| export
def pmu(
    description: str = None,  # Optional, use to give descriptive log for the memory usage at a particular point in the code.
    mode: Literal["both", "cpu", "gpu"] = "both",  # Specify which memory to print. Options are "cpu", "gpu", or "both" (the default).
):
    """Print memory usage
    
    
    """
    if description:
       print(description) 
    
    if mode in ("cpu", "both"):
        main_process = psutil.Process()
        child_processes = main_process.children(recursive=True)
        total_memory_usage = main_process.memory_info().rss
        for child in child_processes:
            total_memory_usage += child.memory_info().rss
        print(f"CPU Memory usage: {total_memory_usage / 1024 ** 2:.2f} MB")
        
    if mode in ("gpu", "both"):
        try:
            from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
        except ImportError as e:
            e.args += "Checking gpu memory is only supported for Nvidia devices and requires the pynvml library to be installed"
            raise e

        nvmlInit()
        handle = nvmlDeviceGetHandleByIndex(0)
        info = nvmlDeviceGetMemoryInfo(handle)
        print("GPU Memory Usage:")
        print(f"Total: {info.total / 1024**2:.2f} MiB")
        print(f"Used: {info.used / 1024**2:.2f} MiB")
        print(f"Free: {info.free / 1024**2:.2f} MiB")

In [ ]:
#| export
class NS:
    """Class to store state of variables in the global namespace
    
    Useful for tracking changes to the global namespace at various points in the code.
    Particularly useful for use in a Jupyter notebook.
    """
    def __init__(self):
        self.initial_variables = copy.copy(globals())

    def __call__(self):
        current_variables = copy.copy(globals())

        added = set(current_variables) - set(self.initial_variables)
        removed = set(self.initial_variables) - set(current_variables)
        modified = {
            key for key in (set(current_variables) & set(self.initial_variables))
            if current_variables[key] != self.initial_variables[key]
        }

        print("Added variables:")
        print(added)
        print("\nRemoved variables:")
        print(removed)
        print("\nModified variables:")
        print(modified)

In [ ]:
#| export
def vprint(message):
    """Give functions has a verbose: bool parameter, and add vprint statements to print messages only when verbose is True"""
    import inspect
    local_vars = inspect.currentframe().f_back.f_locals
    if local_vars.get("verbose", False):
        print(message)

In [ ]:
from rich.console import Console

In [ ]:
#| export
console = Console()
print_ts = time.time()

def sprint(*args, **kwargs) -> None:
    global print_ts
    now = time.time()
    proc = multiprocessing.current_process().name
    if proc == "MainProcess":
        proc = f"[bold]{proc:<16}[/bold]"
    else:
        proc = f"{proc:>16}"
    console.print(f"{proc} [[green bold]{now - print_ts:>5.2f}s[/]]", *args, **kwargs)
    print_ts = now
    
def mem_usage() -> str:
    current, peak = tracemalloc.get_traced_memory()
    return f"Memory usage: {current//1024//1024} MB; peak: {peak//1024//1024} MB"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
def make_data(length: int) -> np.array:
    data = []
    sprint("Putting data into a Python list.", mem_usage()) 
    for i in range(length // 5):
        data.append((datetime.datetime.now(), 0))
        data.append((datetime.datetime.now(), 0))
        data.append((datetime.datetime.now(), 1))
        data.append((datetime.datetime.now(), 0))
        data.append((datetime.datetime.now(), np.nan))
    sprint("Converting into a data frame.", mem_usage())
    df = pd.DataFrame(data, columns=["timestamp", "val"])
    sprint("Converting into numpy array", mem_usage())
    np_array = df.to_records()
    sprint("Returning", mem_usage())
    return np_array

tracemalloc.start()
print_ts = time.time()
make_data(500000)
tracemalloc.stop()


MainProcess      [ 0.00s] Putting data into a Python list. Memory usage: 0 MB; peak: 0 MB

MainProcess      [ 0.73s] Converting into a data frame. Memory usage: 49 MB; peak: 49 MB

MainProcess      [ 0.63s] Converting into numpy array Memory usage: 57 MB; peak: 81 MB

MainProcess      [ 0.01s] Returning Memory usage: 72 MB; peak: 81 MB

In [ ]:
from multiprocessing.managers import SharedMemoryManager

SharedMemoryManager??

Init signature: SharedMemoryManager(*args, **kwargs)
Source:        
    class SharedMemoryManager(BaseManager):
        """Like SyncManager but uses SharedMemoryServer instead of Server.

        It provides methods for creating and returning SharedMemory instances
        and for creating a list-like object (ShareableList) backed by shared
        memory.  It also provides methods that create and return Proxy Objects
        that support synchronization across processes (i.e. multi-process-safe
        locks and semaphores).
        """

        _Server = SharedMemoryServer

        def __init__(self, *args, **kwargs):
            if os.name == "posix":
                # bpo-36867: Ensure the resource_tracker is running before
                # launching the manager process, so that concurrent
                # shared_memory manipulation both in the manager and in the
                # current process does not create two resource_tracker
                # processes.
                f